In [603]:
import pandas as pd  #데이터 파일 관련 보뮬
from sklearn.neighbors import KNeighborsClassifier  #기계 학습 관련 모듈
from matplotlib import pyplot as plt
# from sklearnex import patch_sklearn
# patch_sklearn()
import seaborn as sns

In [604]:
file="../data/bald_probability.csv"

In [605]:
bproDF=pd.read_csv(file)

In [606]:
bproDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7917 entries, 0 to 7916
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            7832 non-null   float64
 1   gender         7840 non-null   object 
 2   job_role       6600 non-null   object 
 3   province       7831 non-null   object 
 4   salary         7843 non-null   float64
 5   is_married     7845 non-null   float64
 6   is_hereditary  7828 non-null   float64
 7   weight         7861 non-null   float64
 8   height         7843 non-null   float64
 9   shampoo        7858 non-null   object 
 10  is_smoker      7847 non-null   float64
 11  education      7847 non-null   object 
 12  stress         7853 non-null   float64
 13  bald_prob      7838 non-null   float64
dtypes: float64(9), object(5)
memory usage: 866.0+ KB


[2] 전처리 과정

In [607]:
# 데이터 중복
bproDF.duplicated().sum()

82

In [608]:
bproDF[~bproDF.duplicated()]

,age,gender,job_role,province,salary,is_married,is_hereditary,weight,height,shampoo,is_smoker,education,stress,bald_prob
0,27.0,female,Government Employee,Bengkulu,7.957453e+06,1.0,0.0,54.315053,170.428542,Pantone,1.0,Bachelor Degree,5.0,0.605974
1,53.0,female,Government Employee,Bandung,7.633003e+06,1.0,0.0,72.873404,165.530097,Pantone,0.0,Bachelor Degree,7.0,0.532860
2,37.0,female,Employee,Bandung,6.637625e+06,1.0,0.0,46.321533,154.599388,Moonsilk,0.0,Bachelor Degree,4.0,0.418442
3,36.0,female,Jobless,Palu,3.624871e+06,1.0,0.0,51.539781,167.340481,Deadbuoy,1.0,Elementary School,9.0,0.804050
4,38.0,male,NaN,Palangkaraya,6.031808e+06,1.0,0.0,60.726909,165.514773,Merpati,1.0,Magister Degree,1.0,0.368371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,52.0,male,Government Employee,Manokwari,4.286318e+06,1.0,0.0,46.893444,169.880171,Deadbuoy,1.0,Senior High School,2.0,0.773141
7831,45.0,male,Government Employee,Tanjungselor,8.646146e+06,1.0,0.0,53.878714,170.122857,Pantone,0.0,Bachelor Degree,2.0,0.604058
7832,24.0,male,Government Employee,Jambi,6.850066e+06,1.0,0.0,64.171294,161.384804,Moonsilk,0.0,Senior High School,8.0,0.502413
7833,29.0,male,Employee,Palangkaraya,1.204960e+07,1.0,0.0,55.217752,179.602130,Pantone,1.0,Senior High School,6.0,0.562664


In [609]:
bproDF[~bproDF.duplicated()].shape

(7835, 14)

In [610]:
bproDF = bproDF[~bproDF.duplicated()]

In [611]:
bproDF.boxplot(figsize = (10, 10))
bproDF.boxplot()

<Axes: >

In [612]:
bproDF.isna().sum()

age                85
gender             75
job_role         1300
province           86
salary             73
is_married         70
is_hereditary      88
weight             56
height             74
shampoo            59
is_smoker          69
education          69
stress             64
bald_prob          78
dtype: int64

데이터를 어떻게 볼 것인가?

[1] gender의 결측치......  => 

In [613]:
bproDF.corr(numeric_only=True)

,age,salary,is_married,is_hereditary,weight,height,is_smoker,stress,bald_prob
age,1.000000,0.009776,0.350982,-0.012875,0.002427,-0.007828,0.006622,-0.024732,0.354636
salary,0.009776,1.000000,-0.014108,-0.003883,0.006903,0.012394,-0.005820,0.016902,0.058562
is_married,0.350982,-0.014108,1.000000,-0.003744,0.006765,0.006726,-0.005185,-0.014161,0.125576
is_hereditary,-0.012875,-0.003883,-0.003744,1.000000,0.007816,0.000641,-0.015100,-0.004490,0.439886
weight,0.002427,0.006903,0.006765,0.007816,1.000000,0.394286,0.000074,-0.013385,-0.000688
height,-0.007828,0.012394,0.006726,0.000641,0.394286,1.000000,0.001024,0.005346,0.006229
is_smoker,0.006622,-0.005820,-0.005185,-0.015100,0.000074,0.001024,1.000000,-0.014063,0.264001
stress,-0.024732,0.016902,-0.014161,-0.004490,-0.013385,0.005346,-0.014063,1.000000,0.289476
bald_prob,0.354636,0.058562,0.125576,0.439886,-0.000688,0.006229,0.264001,0.289476,1.000000


In [614]:
bproDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7835 entries, 0 to 7834
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            7750 non-null   float64
 1   gender         7760 non-null   object 
 2   job_role       6535 non-null   object 
 3   province       7749 non-null   object 
 4   salary         7762 non-null   float64
 5   is_married     7765 non-null   float64
 6   is_hereditary  7747 non-null   float64
 7   weight         7779 non-null   float64
 8   height         7761 non-null   float64
 9   shampoo        7776 non-null   object 
 10  is_smoker      7766 non-null   float64
 11  education      7766 non-null   object 
 12  stress         7771 non-null   float64
 13  bald_prob      7757 non-null   float64
dtypes: float64(9), object(5)
memory usage: 918.2+ KB


In [615]:
bproDF.value_counts()

age   gender  job_role             province      salary        is_married  is_hereditary  weight     height      shampoo          is_smoker  education           stress  bald_prob
2.0   male    Government Employee  Kendari       1.150700e+07  0.0         0.0            63.946479  178.733256  Merpati          1.0        Doctoral Degree     3.0     0.454950     1
44.0  male    Employee             Denpasar      5.398613e+06  1.0         1.0            45.636652  154.206293  Merpati          1.0        Senior High School  4.0     0.720523     1
                                                 4.882454e+06  1.0         0.0            48.380036  177.378449  Deadbuoy         1.0        Magister Degree     6.0     0.597222     1
                                   Bengkulu      6.842743e+06  1.0         0.0            47.761236  175.948554  Pantone          0.0        Bachelor Degree     9.0     0.514561     1
                                   Banjarmasin   1.143873e+07  1.0         0.0       

In [616]:
bproDF

,age,gender,job_role,province,salary,is_married,is_hereditary,weight,height,shampoo,is_smoker,education,stress,bald_prob
0,27.0,female,Government Employee,Bengkulu,7.957453e+06,1.0,0.0,54.315053,170.428542,Pantone,1.0,Bachelor Degree,5.0,0.605974
1,53.0,female,Government Employee,Bandung,7.633003e+06,1.0,0.0,72.873404,165.530097,Pantone,0.0,Bachelor Degree,7.0,0.532860
2,37.0,female,Employee,Bandung,6.637625e+06,1.0,0.0,46.321533,154.599388,Moonsilk,0.0,Bachelor Degree,4.0,0.418442
3,36.0,female,Jobless,Palu,3.624871e+06,1.0,0.0,51.539781,167.340481,Deadbuoy,1.0,Elementary School,9.0,0.804050
4,38.0,male,NaN,Palangkaraya,6.031808e+06,1.0,0.0,60.726909,165.514773,Merpati,1.0,Magister Degree,1.0,0.368371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,52.0,male,Government Employee,Manokwari,4.286318e+06,1.0,0.0,46.893444,169.880171,Deadbuoy,1.0,Senior High School,2.0,0.773141
7831,45.0,male,Government Employee,Tanjungselor,8.646146e+06,1.0,0.0,53.878714,170.122857,Pantone,0.0,Bachelor Degree,2.0,0.604058
7832,24.0,male,Government Employee,Jambi,6.850066e+06,1.0,0.0,64.171294,161.384804,Moonsilk,0.0,Senior High School,8.0,0.502413
7833,29.0,male,Employee,Palangkaraya,1.204960e+07,1.0,0.0,55.217752,179.602130,Pantone,1.0,Senior High School,6.0,0.562664


In [617]:
bproDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7835 entries, 0 to 7834
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            7750 non-null   float64
 1   gender         7760 non-null   object 
 2   job_role       6535 non-null   object 
 3   province       7749 non-null   object 
 4   salary         7762 non-null   float64
 5   is_married     7765 non-null   float64
 6   is_hereditary  7747 non-null   float64
 7   weight         7779 non-null   float64
 8   height         7761 non-null   float64
 9   shampoo        7776 non-null   object 
 10  is_smoker      7766 non-null   float64
 11  education      7766 non-null   object 
 12  stress         7771 non-null   float64
 13  bald_prob      7757 non-null   float64
dtypes: float64(9), object(5)
memory usage: 918.2+ KB


[1-1] 결측치 확인


In [618]:
bproDF_na=bproDF.isnull().sum()
bproDF_na

age                85
gender             75
job_role         1300
province           86
salary             73
is_married         70
is_hereditary      88
weight             56
height             74
shampoo            59
is_smoker          69
education          69
stress             64
bald_prob          78
dtype: int64

In [619]:


for i in bproDF[bproDF['gender'].isnull()].index:
    if bproDF['height'][i] >= 158:
        bproDF.loc[i, 'gender'] = 'male'
    else:
        bproDF.loc[i, 'gender'] = 'female'

print(bproDF)

       age  gender             job_role      province        salary  \
0     27.0  female  Government Employee      Bengkulu  7.957453e+06   
1     53.0  female  Government Employee       Bandung  7.633003e+06   
2     37.0  female             Employee       Bandung  6.637625e+06   
3     36.0  female              Jobless          Palu  3.624871e+06   
4     38.0    male                  NaN  Palangkaraya  6.031808e+06   
...    ...     ...                  ...           ...           ...   
7830  52.0    male  Government Employee     Manokwari  4.286318e+06   
7831  45.0    male  Government Employee  Tanjungselor  8.646146e+06   
7832  24.0    male  Government Employee         Jambi  6.850066e+06   
7833  29.0    male             Employee  Palangkaraya  1.204960e+07   
7834  42.0  female             Employee       Kendari  4.888239e+06   

      is_married  is_hereditary     weight      height   shampoo  is_smoker  \
0            1.0            0.0  54.315053  170.428542   Pantone    

In [620]:
bproDF_na

age                85
gender             75
job_role         1300
province           86
salary             73
is_married         70
is_hereditary      88
weight             56
height             74
shampoo            59
is_smoker          69
education          69
stress             64
bald_prob          78
dtype: int64

In [621]:
bproDF['height'][0]

170.42854234879383

In [622]:
#오우...많다.

#결측치 처리하기
#나이 컬럼에 평균 값 집어넣기

In [623]:
mean_age=bproDF['age'].mean()
bproDF['age'].fillna(round(mean_age),inplace=True)

In [624]:
# 성별 부분에서 중요 값 채워넣기 (mode)
# mode_gender=bproDF['gender'].mode().values[0]
# bproDF['gender'].fillna(mode_gender,inplace=True)
for i in bproDF[bproDF['gender'].isnull()].index:
    if bproDF['height'][i] >= 158:
        bproDF.loc[i, 'gender'] = 'male'
    else:
        bproDF.loc[i, 'gender'] = 'female'

In [625]:
mode_edu=bproDF['education'].mode().values[0]
bproDF['education'].fillna(mode_edu,inplace=True)

In [626]:
mode_shampoo=bproDF['shampoo'].mode().values[0]
bproDF['shampoo'].fillna(mode_shampoo,inplace=True)

In [627]:
#텅 빈 값 알아서 넣어주기

In [628]:
bproDF['job_role'].fillna('Unspecified',inplace=True)

In [629]:
bproDF['province'].fillna('Others',inplace=True)

In [630]:
#평균 집어넣기

In [631]:
mean_salary=bproDF['salary'].mean()
bproDF['salary'].fillna(mean_salary,inplace=True)

In [632]:
mean_married=bproDF['is_married'].mean()
bproDF['is_married'].fillna(round(mean_married),inplace=True)

In [633]:
mean_hereditary=bproDF['is_hereditary'].mean()
bproDF['is_hereditary'].fillna(round(mean_hereditary),inplace=True)

In [634]:
mean_weight=bproDF['weight'].mean()
bproDF['weight'].fillna(mean_weight,inplace=True)

In [635]:
mean_height=bproDF['height'].mean()
bproDF['height'].fillna(mean_height,inplace=True)

In [636]:
mean_smoke=bproDF['is_smoker'].mean()
bproDF['is_smoker'].fillna(round(mean_smoke),inplace=True)

In [637]:
mean_stress=bproDF['stress'].mean()
bproDF['stress'].fillna(round(mean_stress),inplace=True)

In [638]:
# bald_probability에서 null 제거하기
bproDF=bproDF[~bproDF['bald_prob'].isnull()]


In [639]:
bproDF_na=bproDF.isnull().sum()

In [640]:
bproDF_na


age              0
gender           0
job_role         0
province         0
salary           0
is_married       0
is_hereditary    0
weight           0
height           0
shampoo          0
is_smoker        0
education        0
stress           0
bald_prob        0
dtype: int64

In [641]:
bproDF

,age,gender,job_role,province,salary,is_married,is_hereditary,weight,height,shampoo,is_smoker,education,stress,bald_prob
0,27.0,female,Government Employee,Bengkulu,7.957453e+06,1.0,0.0,54.315053,170.428542,Pantone,1.0,Bachelor Degree,5.0,0.605974
1,53.0,female,Government Employee,Bandung,7.633003e+06,1.0,0.0,72.873404,165.530097,Pantone,0.0,Bachelor Degree,7.0,0.532860
2,37.0,female,Employee,Bandung,6.637625e+06,1.0,0.0,46.321533,154.599388,Moonsilk,0.0,Bachelor Degree,4.0,0.418442
3,36.0,female,Jobless,Palu,3.624871e+06,1.0,0.0,51.539781,167.340481,Deadbuoy,1.0,Elementary School,9.0,0.804050
4,38.0,male,Unspecified,Palangkaraya,6.031808e+06,1.0,0.0,60.726909,165.514773,Merpati,1.0,Magister Degree,1.0,0.368371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,52.0,male,Government Employee,Manokwari,4.286318e+06,1.0,0.0,46.893444,169.880171,Deadbuoy,1.0,Senior High School,2.0,0.773141
7831,45.0,male,Government Employee,Tanjungselor,8.646146e+06,1.0,0.0,53.878714,170.122857,Pantone,0.0,Bachelor Degree,2.0,0.604058
7832,24.0,male,Government Employee,Jambi,6.850066e+06,1.0,0.0,64.171294,161.384804,Moonsilk,0.0,Senior High School,8.0,0.502413
7833,29.0,male,Employee,Palangkaraya,1.204960e+07,1.0,0.0,55.217752,179.602130,Pantone,1.0,Senior High School,6.0,0.562664


In [642]:
cor_bald=bproDF.corr(numeric_only=True)

In [643]:
cor_bald

,age,salary,is_married,is_hereditary,weight,height,is_smoker,stress,bald_prob
age,1.000000,0.010703,0.347367,-0.011338,0.002739,-0.006100,0.007164,-0.026499,0.351822
salary,0.010703,1.000000,-0.013812,-0.003866,0.009862,0.013850,-0.005561,0.015103,0.058254
is_married,0.347367,-0.013812,1.000000,-0.003135,0.006382,0.008667,-0.004346,-0.015405,0.124883
is_hereditary,-0.011338,-0.003866,-0.003135,1.000000,0.006703,0.003332,-0.014558,-0.003764,0.437696
weight,0.002739,0.009862,0.006382,0.006703,1.000000,0.389920,-0.001635,-0.012359,-0.000685
height,-0.006100,0.013850,0.008667,0.003332,0.389920,1.000000,-0.000818,0.005741,0.006200
is_smoker,0.007164,-0.005561,-0.004346,-0.014558,-0.001635,-0.000818,1.000000,-0.013842,0.261128
stress,-0.026499,0.015103,-0.015405,-0.003764,-0.012359,0.005741,-0.013842,1.000000,0.288255
bald_prob,0.351822,0.058254,0.124883,0.437696,-0.000685,0.006200,0.261128,0.288255,1.000000


In [644]:
plt.figure(figsize=(10,10))
sns.heatmap(data =cor_bald, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

<Axes: >

In [645]:
bproDF.columns

Index(['age', 'gender', 'job_role', 'province', 'salary', 'is_married',
       'is_hereditary', 'weight', 'height', 'shampoo', 'is_smoker',
       'education', 'stress', 'bald_prob'],
      dtype='object')

In [646]:

df_err = bproDF[bproDF['age'] < 10]
bproDF = bproDF[~bproDF['age'].isin(df_err['age'])]


bproDF['age_range'] = bproDF['age'].apply(lambda x: '0+' if x < 10
                                  else '10+' if 10 <= x < 20
                                  else '20+' if 20 <= x < 30
                                  else '30+' if 30 <= x < 40
                                  else '40+' if 40 <= x < 50
                                  else '50+' if 50 <= x < 60
                                  else '60+' if 60 <= x < 70
                                  else '70+' if 70 <= x < 80
                                  else '80+')


bproDF['salary_range'] = bproDF['salary'].apply(lambda x: 'low' if x < 3070000
                                        else 'middle' if 3070000 <= x < 8000000
                                        else 'high') 
                                        


bproDF['stress_range'] = bproDF['stress'].apply(lambda x: "Level 1" if x <= 3
                                                     else "Level 2" if 4 <= x <= 6
                                                     else "Level 3" if 7 <= x <= 9
                                                     else "Level 4")


bproDF['probability'] =bproDF['bald_prob'].apply(lambda x: 'Low' if x <= 0.5
                                        else 'High')


bproDF['marriage_status'] = bproDF['is_married'].apply(lambda x: 'Yes' if x == 1 else 'No')
bproDF['smoking'] = bproDF['is_smoker'].apply(lambda x: 'Yes' if x == 1 else 'No')
bproDF['family_hereditary'] = bproDF['is_hereditary'].apply(lambda x: 'Yes' if x == 1 else 'No')
bproDF['BMI']=(bproDF['weight']/bproDF['height'])**2
bproDF['obesity type']=bproDF['BMI'].apply(lambda x:'low weight' if x<18.5 else "normal weight" if 18.5<=x<23 else "high weight" if 23<=x<25 else "obesity" if 25<=x<40 else "very obesity")
                                             
                                               
                                               
                                              

bproDF.columns

C:\Users\kdp\AppData\Local\Temp\ipykernel_1332\4893508.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bproDF['age_range'] = bproDF['age'].apply(lambda x: '0+' if x < 10
C:\Users\kdp\AppData\Local\Temp\ipykernel_1332\4893508.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bproDF['salary_range'] = bproDF['salary'].apply(lambda x: 'low' if x < 3070000
C:\Users\kdp\AppData\Local\Temp\ipykernel_1332\4893508.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

Index(['age', 'gender', 'job_role', 'province', 'salary', 'is_married',
       'is_hereditary', 'weight', 'height', 'shampoo', 'is_smoker',
       'education', 'stress', 'bald_prob', 'age_range', 'salary_range',
       'stress_range', 'probability', 'marriage_status', 'smoking',
       'family_hereditary', 'BMI', 'obesity type'],
      dtype='object')

In [647]:
bproDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7750 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                7750 non-null   float64
 1   gender             7750 non-null   object 
 2   job_role           7750 non-null   object 
 3   province           7750 non-null   object 
 4   salary             7750 non-null   float64
 5   is_married         7750 non-null   float64
 6   is_hereditary      7750 non-null   float64
 7   weight             7750 non-null   float64
 8   height             7750 non-null   float64
 9   shampoo            7750 non-null   object 
 10  is_smoker          7750 non-null   float64
 11  education          7750 non-null   object 
 12  stress             7750 non-null   float64
 13  bald_prob          7750 non-null   float64
 14  age_range          7750 non-null   object 
 15  salary_range       7750 non-null   object 
 16  stress_range       7750 non-n

In [648]:
bproDF.corr(numeric_only=True)

,age,salary,is_married,is_hereditary,weight,height,is_smoker,stress,bald_prob,BMI
age,1.000000,0.011377,0.335560,-0.012923,0.003425,-0.005570,0.008551,-0.027253,0.351014,0.008295
salary,0.011377,1.000000,-0.012801,-0.003772,0.009982,0.014072,-0.005680,0.014862,0.058547,0.002315
is_married,0.335560,-0.012801,1.000000,-0.006400,0.007821,0.010043,-0.001654,-0.016736,0.121769,0.006717
is_hereditary,-0.012923,-0.003772,-0.006400,1.000000,0.006798,0.003418,-0.014366,-0.003837,0.437712,0.005687
weight,0.003425,0.009982,0.007821,0.006798,1.000000,0.389833,-0.001583,-0.012058,-0.000390,0.922347
height,-0.005570,0.014072,0.010043,0.003418,0.389833,1.000000,-0.001122,0.005670,0.005765,0.022041
is_smoker,0.008551,-0.005680,-0.001654,-0.014366,-0.001583,-0.001122,1.000000,-0.013973,0.261299,-0.001942
stress,-0.027253,0.014862,-0.016736,-0.003837,-0.012058,0.005670,-0.013973,1.000000,0.287893,-0.015880
bald_prob,0.351014,0.058547,0.121769,0.437712,-0.000390,0.005765,0.261299,0.287893,1.000000,-0.001859
BMI,0.008295,0.002315,0.006717,0.005687,0.922347,0.022041,-0.001942,-0.015880,-0.001859,1.000000


In [649]:
bproDF.corr(numeric_only=True,method='spearman')

,age,salary,is_married,is_hereditary,weight,height,is_smoker,stress,bald_prob,BMI
age,1.000000,-0.001310,0.238853,-0.010720,0.003286,-0.003074,0.008905,-0.024912,0.362501,0.004180
salary,-0.001310,1.000000,-0.008653,0.002579,0.006602,0.011045,-0.004560,0.018101,0.049189,0.001502
is_married,0.238853,-0.008653,1.000000,-0.006400,0.002722,0.008960,-0.001654,-0.016563,0.122046,-0.002011
is_hereditary,-0.010720,0.002579,-0.006400,1.000000,0.008158,0.004979,-0.014366,-0.003599,0.428164,0.005381
weight,0.003286,0.006602,0.002722,0.008158,1.000000,0.335645,0.003107,-0.017302,0.002086,0.887862
height,-0.003074,0.011045,0.008960,0.004979,0.335645,1.000000,-0.000161,0.006889,0.009497,-0.087636
is_smoker,0.008905,-0.004560,-0.001654,-0.014366,0.003107,-0.000161,1.000000,-0.013624,0.263648,0.006357
stress,-0.024912,0.018101,-0.016563,-0.003599,-0.017302,0.006889,-0.013624,1.000000,0.286831,-0.019221
bald_prob,0.362501,0.049189,0.122046,0.428164,0.002086,0.009497,0.263648,0.286831,1.000000,-0.002594
BMI,0.004180,0.001502,-0.002011,0.005381,0.887862,-0.087636,0.006357,-0.019221,-0.002594,1.000000


In [650]:
bproDF.corr(numeric_only=True,method='kendall')

,age,salary,is_married,is_hereditary,weight,height,is_smoker,stress,bald_prob,BMI
age,1.000000,-0.000945,0.197828,-0.008879,0.002257,-0.002099,0.007376,-0.017694,0.249344,0.002813
salary,-0.000945,1.000000,-0.007066,0.002106,0.004357,0.007304,-0.003724,0.012696,0.032834,0.000916
is_married,0.197828,-0.007066,1.000000,-0.006400,0.002223,0.007316,-0.001654,-0.014184,0.099664,-0.001642
is_hereditary,-0.008879,0.002106,-0.006400,1.000000,0.006662,0.004065,-0.014366,-0.003083,0.349645,0.004394
weight,0.002257,0.004357,0.002223,0.006662,1.000000,0.228012,0.002537,-0.012092,0.001439,0.709994
height,-0.002099,0.007304,0.007316,0.004065,0.228012,1.000000,-0.000131,0.004796,0.006333,-0.062030
is_smoker,0.007376,-0.003724,-0.001654,-0.014366,0.002537,-0.000131,1.000000,-0.011667,0.215299,0.005191
stress,-0.017694,0.012696,-0.014184,-0.003083,-0.012092,0.004796,-0.011667,1.000000,0.201877,-0.013483
bald_prob,0.249344,0.032834,0.099664,0.349645,0.001439,0.006333,0.215299,0.201877,1.000000,-0.001674
BMI,0.002813,0.000916,-0.001642,0.004394,0.709994,-0.062030,0.005191,-0.013483,-0.001674,1.000000


In [651]:
#BMI 와 탈모 간의 상관관계

In [652]:
corr_BMI = bproDF['bald_prob'].corr(bproDF['BMI'], method='pearson')

In [653]:
bproDF

,age,gender,job_role,province,salary,is_married,is_hereditary,weight,height,shampoo,...,bald_prob,age_range,salary_range,stress_range,probability,marriage_status,smoking,family_hereditary,BMI,obesity type
0,27.0,female,Government Employee,Bengkulu,7.957453e+06,1.0,0.0,54.315053,170.428542,Pantone,...,0.605974,20+,middle,Level 2,High,Yes,Yes,No,0.101568,low weight
1,53.0,female,Government Employee,Bandung,7.633003e+06,1.0,0.0,72.873404,165.530097,Pantone,...,0.532860,50+,middle,Level 3,High,Yes,No,No,0.193814,low weight
2,37.0,female,Employee,Bandung,6.637625e+06,1.0,0.0,46.321533,154.599388,Moonsilk,...,0.418442,30+,middle,Level 2,Low,Yes,No,No,0.089774,low weight
3,36.0,female,Jobless,Palu,3.624871e+06,1.0,0.0,51.539781,167.340481,Deadbuoy,...,0.804050,30+,middle,Level 3,High,Yes,Yes,No,0.094860,low weight
4,38.0,male,Unspecified,Palangkaraya,6.031808e+06,1.0,0.0,60.726909,165.514773,Merpati,...,0.368371,30+,middle,Level 1,Low,Yes,Yes,No,0.134614,low weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7830,52.0,male,Government Employee,Manokwari,4.286318e+06,1.0,0.0,46.893444,169.880171,Deadbuoy,...,0.773141,50+,middle,Level 1,High,Yes,Yes,No,0.076197,low weight
7831,45.0,male,Government Employee,Tanjungselor,8.646146e+06,1.0,0.0,53.878714,170.122857,Pantone,...,0.604058,40+,high,Level 1,High,Yes,No,No,0.100302,low weight
7832,24.0,male,Government Employee,Jambi,6.850066e+06,1.0,0.0,64.171294,161.384804,Moonsilk,...,0.502413,20+,middle,Level 3,High,Yes,No,No,0.158109,low weight
7833,29.0,male,Employee,Palangkaraya,1.204960e+07,1.0,0.0,55.217752,179.602130,Pantone,...,0.562664,20+,high,Level 2,High,Yes,Yes,No,0.094522,low weight


In [654]:
# (3) 기계학습 데이터셋 준비 => 피쳐와 타겟 분리

In [655]:
#모듈 임포트 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,classification_report,confusion_matrix

select=['age_range','stress_range','is_married','is_smoker','family_hereditary']

In [656]:
# 상관관계 없는 피쳐들을 잘 못 상관관계를 나타내는 일이 없도록 dummies 메서드를 사용한다.


In [657]:
dummies=pd.get_dummies(bproDF[select])

In [658]:
feature=bproDF[['age', 'stress', 'is_married', 'is_smoker','is_hereditary']]
target=bproDF['probability']

In [659]:
# 그래프로 산포도 그려보기
def print_feature(nrows, ncols, df, target, features, corrs):
    plt.figure(figsize=(12, 8))  # 새로운 그림 생성
    
    for idx in range(len(features)):
        plt.subplot(nrows, ncols, idx + 1)
        plt.scatter(df[target], df[features[idx]], label=f'corr:{corrs[idx]:.2f}')
        plt.legend()
        plt.xlabel(target)
        plt.ylabel(features[idx])
    
    plt.tight_layout()
    plt.show()

In [660]:
corrList=bproDF.corr(numeric_only=True)['bald_prob'].to_list()


In [661]:
corrList

[0.3510138530113167,
 0.05854714538978923,
 0.12176901772660158,
 0.43771217794286843,
 -0.00039035393429623025,
 0.005765177133962315,
 0.26129872934035864,
 0.28789267047159944,
 1.0,
 -0.001858689682609019]

In [662]:
# print_feature(10,10,bproDF,target,feature,)

In [663]:
#트레인과 테스트로 분할


In [664]:
X_train, X_test, y_train, y_test = train_test_split(feature,target,test_size=0.2,random_state=41)

In [665]:
#스케일링 

In [666]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit_transform(X_train)

X_train_sc=sc.transform(X_train)
X_test=sc.transform(X_test)

[모델 학습] 

[1-1] LogisticRegression

In [667]:
LR_model=LogisticRegression()

LR_model.fit(X_train_sc,y_train)


LogisticRegression()

In [668]:
#LR 모델의 스코어 내기
LR_train_score=LR_model.score(X_train_sc,y_train)
LR_test_score=LR_model.score(X_test,y_test)

In [669]:
print(f'Training score: {LR_train_score}', f'Test score: {LR_test_score}')

Training score: 0.7691935483870967 Test score: 0.7683870967741936


In [670]:
#예측: predict 사용

In [671]:
#확률도 같이
y_pred=LR_model.predict(X_test)
y_pred

array(['High', 'Low', 'Low', ..., 'High', 'High', 'High'], dtype=object)

In [672]:
y_prob=LR_model.predict_proba(X_test)
y_prob

array([[0.79059759, 0.20940241],
       [0.15209746, 0.84790254],
       [0.07947315, 0.92052685],
       ...,
       [0.81280923, 0.18719077],
       [0.92837111, 0.07162889],
       [0.92229136, 0.07770864]])

In [673]:
#정확도
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.7683870967741936

In [674]:
# # #재현율
# precision=precision_score(y_test,y_pred)
#  # 안됨......

In [675]:
result_predict=classification_report(y_test,y_pred)

print(f'결과표:\n{result_predict}')


결과표:
              precision    recall  f1-score   support

        High       0.80      0.86      0.83      1003
         Low       0.70      0.60      0.64       547

    accuracy                           0.77      1550
   macro avg       0.75      0.73      0.74      1550
weighted avg       0.76      0.77      0.76      1550


[1-2] DecisionTree model 

In [676]:
#DecisionTreeClassifier 인스턴스
from sklearn.tree import DecisionTreeClassifier
dt_model=DecisionTreeClassifier(random_state=23)
dt_model.fit(X_train_sc,y_train)

DecisionTreeClassifier(random_state=23)

[2-3] SVC 모델

In [677]:
from sklearn.svm import SVC
svc_model=SVC(probability=True)
svc_model.fit(X_train_sc,y_train)

SVC(probability=True)

[2-4] 랜덤 포레스트

In [678]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(random_state=5)
rf_model.fit(X_train_sc,y_train)

RandomForestClassifier(random_state=5)

[2-5] lightbgm


In [679]:
import lightgbm

print(lightgbm.__version__)

4.1.0


[3] 3개의 모델 한꺼번에 비교( 같은 데이터셋을 이용한 비교)

In [680]:
# X_train, y_train을 다시 쪼개서 90%는 학습과 10%는 검증용 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, 
                                           y_train, 
                                           test_size=0.1, 
                                           random_state=156 )


In [681]:
# XGBoost와 동일하게 n_estimators는 400 설정.
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(n_estimators=400, learning_rate=0.05)


In [682]:
# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능
evals = [(X_tr, y_tr), (X_val, y_val)]

lgbm_wrapper.set_params(early_stopping_rounds=50)
lgbm_wrapper.fit(X_tr, y_tr, 
                 eval_metric="logloss", 
                 eval_set=evals)

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Number of positive: 1948, number of negative: 3632
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000125 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 5580, number of used features: 5
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.349104 -> initscore=-0.622980
[LightGBM] [Info] Start training from score -0.622980
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[49]	training's binary_logloss: 0.433148	valid_1's binary_logloss: 0.471193


LGBMClassifier(early_stopping_rounds=50, learning_rate=0.05, n_estimators=400)

In [683]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [684]:
#확률 보기
preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

In [685]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)

    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print(f'정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1: {f1:.4f}, AUC:{4:.4f}')

In [686]:
# get_clf_eval(y_test, preds, pred_proba)

In [687]:
from sklearn.ensemble import VotingClassifier
voting_model=VotingClassifier(estimators=[('LR_model',LR_model),('dt_model',dt_model),('svc_model',svc_model),('rf_model',rf_model)],verbose=True)

In [688]:
voting_model.fit(X_train_sc,y_train)

[Voting] ................. (1 of 4) Processing LR_model, total=   0.0s
[Voting] ................. (2 of 4) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 4) Processing svc_model, total=   6.6s
[Voting] ................. (4 of 4) Processing rf_model, total=   0.6s


VotingClassifier(estimators=[('LR_model', LogisticRegression()),
                             ('dt_model',
                              DecisionTreeClassifier(random_state=23)),
                             ('svc_model', SVC(probability=True)),
                             ('rf_model',
                              RandomForestClassifier(random_state=5))],
                 verbose=True)

In [689]:
#이 부분 잘 모르겠음
# new_data=pd.DataFrame([X_test.iloc[0]],columns=X_test.columns)
# new_data

In [690]:
print(f'[LogisticRegression] Train score: {LR_model.score(X_train_sc,y_train)},Test score: {LR_model.score(X_test,y_test)}')

print(f'[DecisionTreeClassifier] Train score: {dt_model.score(X_train_sc,y_train)},Test score: {dt_model.score(X_test,y_test)}')

print(f'[RandomForestClassifier] Train score: {svc_model.score(X_train_sc,y_train)},Test score: {svc_model.score(X_test,y_test)}')

[LogisticRegression] Train score: 0.7691935483870967,Test score: 0.7683870967741936
[DecisionTreeClassifier] Train score: 0.8187096774193549,Test score: 0.7561290322580645
[RandomForestClassifier] Train score: 0.7733870967741936,Test score: 0.7761290322580645


In [691]:
# 1. 금융 및 보험 서비스 finance work: 518만 루피아
# 2. 정보통신 infomation : 505만 루피아
# 3. 채광 및 채석 mining: 481만 루피아
# 4. 전기 및 가스 조달 electronic&gas: 449만 루피아
# 5. 부동산 property: 442만 루피아
# 6. 기업 서비스 company service: 393만 루피아
# 7. 정부 행정 government administration: 385만 루피아
# 8. 건강 서비스 및 사회 활동 health care:
# 362만 루피아
# 9. 운송 및 창고 transport& ware house: 360만 루피아


In [692]:
# 모듈 로딩
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

# 모델 조합 조건
# CV => 기본 5,..
# DT의 하이퍼파라미터 => max_depth,gini.....
dt_model = DecisionTreeClassifier()
n_cv =5
params={'criterion':['gini','entropy','log_loss'],
        'max_depth':[5,10,15],
        'min_samples_split':[2,4,6]
        }
# 하이퍼파라미터 조합된 모델 생성 후 교차검증 데이터셋으로 학습
#시켜주는 인스턴스
gs=GridSearchCV(dt_model,param_grid=params,cv=n_cv)

In [693]:
# 조합된 모델들의 학습 진행
result=gs.fit(X_train_sc,y_train)

In [694]:
X_train_sc.shape

(6200, 5)

In [695]:
# 결과 확인 
result

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10, 15],
                         'min_samples_split': [2, 4, 6]})

In [696]:
# 학습 후 저장된 모델 파라미터 확인 
#교차 학습의 결과 알려주는 코드
gs.cv_results_
#보기 불편해서 데이터 프레임에 담기
cv_resultsDF=pd.DataFrame(gs.cv_results_)

In [697]:
cv_resultsDF

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008237,0.007544,0.000368,0.000451,gini,5,2,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.762097,0.754839,0.811290,0.777419,0.778226,0.776774,0.019445,1
1,0.005170,0.006687,0.005012,0.009437,gini,5,4,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.762097,0.754839,0.811290,0.777419,0.778226,0.776774,0.019445,1
2,0.003616,0.005258,0.003006,0.005058,gini,5,6,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",0.762097,0.754839,0.811290,0.777419,0.778226,0.776774,0.019445,1
3,0.013117,0.006585,0.000000,0.000000,gini,10,2,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.766935,0.747581,0.770968,0.766129,0.760484,0.762419,0.008138,12
4,0.007180,0.007933,0.002826,0.005652,gini,10,4,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.767742,0.749194,0.773387,0.771774,0.762097,0.764839,0.008739,10
5,0.006026,0.007839,0.003506,0.006139,gini,10,6,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.768548,0.749194,0.771774,0.771774,0.762903,0.764839,0.008467,11
6,0.011668,0.005097,0.001483,0.000995,gini,15,2,"{'criterion': 'gini', 'max_depth': 15, 'min_sa...",0.759677,0.740323,0.754839,0.745968,0.750806,0.750323,0.006740,25
7,0.006249,0.007654,0.003138,0.006277,gini,15,4,"{'criterion': 'gini', 'max_depth': 15, 'min_sa...",0.761290,0.740323,0.757258,0.750806,0.752419,0.752419,0.007086,22
8,0.006373,0.007808,0.000000,0.000000,gini,15,6,"{'criterion': 'gini', 'max_depth': 15, 'min_sa...",0.760484,0.741935,0.757258,0.750000,0.755645,0.753065,0.006520,19
9,0.003123,0.006246,0.003141,0.006283,entropy,5,2,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.762903,0.747581,0.812903,0.778226,0.779032,0.776129,0.021703,4


In [698]:
# 조합된 모델 중 최고의 성능을 내는 하이퍼파라미터
gs.best_params_

{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2}

In [699]:
# 조합된 모델 중 최고의 성능을 내는 모델 인스턴스
my_best_model=gs.best_estimator_

In [700]:
#조합된 모델 중 최고의 점수
gs.best_score_  #테스트 스코어가 기준, 만약 트레인 스코어가 보고 싶으면, return_train_score 부분을 False로 해준다.

0.7767741935483871

[6] 학습 및 튜닝 후 모델 저장 <hr>
-다른 파일에서 활용이 가능한 파일 형태로 저장
-웹/앱 원하는 곳에서 활용가능
-scikit-learn에서는 pickle 포맷으로 저장
-joblib 모듈에 해당 기능 포함되어 있음

In [701]:
import joblib 
import os

In [702]:
# 모델 저장
model_dir='../model/'
model_filename=model_dir+'bald_dt.pkl'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)


In [703]:
joblib.dump(my_best_model,model_filename)

['../model/bald_dt.pkl']